# Meridonal Overturning circulation
- calculations may be improved by taking partial bottom cells
- possibly SSH adds accuracy, but it is centered on the T-grid

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - matplotlib.pyplot

In [ ]:
from MOC import calculate_AMOC, AMOC_max, approx_lats
from grid import create_tdepth
from paths import file_ex_ocn_ctrl, path_results
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_DZ, xr_DXU

In [ ]:
# %%time
# for i, (y,m,s) in enumerate(IterateOutputCESM(domain='ocn', run='rcp', tavg='yrly', name='UVEL_VVEL')):
#     # ca. 20 per year
#     print(i, y, s)
#     ds = xr.open_dataset(s, decode_times=False)
#     AMOC = calculate_AMOC(ds)
#     if i==0:
#         AMOC_out = AMOC.copy()
#     else:
#         AMOC_out = xr.concat([AMOC_out, AMOC], dim='time')
# #     if y==202: break
# AMOC_out.to_netcdf(f'{path_results}/MOC/AMOC_rcp.nc')

In [ ]:
AMOC_ctrl = xr.open_dataarray(f'{path_results}/MOC/AMOC_ctrl.nc')
AMOC_rcp  = xr.open_dataarray(f'{path_results}/MOC/AMOC_rcp.nc' )

In [ ]:
tdepth = create_tdepth('ocn')
lats = approx_lats('ocn')
X, Y = np.meshgrid(lats, -tdepth/1e3)


f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)

plt.pcolormesh(X, Y, AMOC_ctrl.mean(dim='time')/1e6, vmin=-20, vmax=20, cmap='RdBu_r')
plt.text(.05, .9, r'$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='max')
cbar.ax.set_ylabel('AMOC volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_depth_lat_ctrl_mean')

    
f, ax = plt.subplots(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.pcolormesh(X, Y, (AMOC_rcp.mean(dim='time')-AMOC_ctrl.mean(dim='time'))/1e6, vmin=-2, vmax=2, cmap='RdBu_r')
plt.text(.05, .9, r'$\langle$RCP$\rangle$-$\langle$CTRL$\rangle$', fontsize=16, transform=ax.transAxes)
plt.xlabel(r'latitude [$^\circ$N]', fontsize=16)
plt.ylabel('depth [km]', fontsize=16)
cbar = plt.colorbar(extend='min')
cbar.ax.set_ylabel('AMOC volume transport [Sv]', fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_depth_lat_rcp-ctrl')

In [ ]:
f = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot(AMOC_max(AMOC_ctrl)/1e6, lw=2, label='CTRL')
plt.plot(AMOC_max(AMOC_rcp)/1e6, lw=2, label='RCP' )
plt.text(.05, .05, '26$^\circ$N, 1000 m', fontsize=16, transform=ax.transAxes)
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('AMOC strength [Sv]', fontsize=16)
plt.legend(fontsize=16, frameon=False)
plt.tight_layout()
plt.savefig(f'{path_results}/MOC/AMOC_timeseries')